# Sora2 Watermark Remover (Local Jupyter版)

このノートブックは、Sora動画からウォーターマークを除去するツールのローカルJupyter版です。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. セットアップ

### ⚠️ 重要: Pillow依存関係の修正

**以下のセルを実行し、その後必ずカーネルを再起動してください。**

1. 下のセルを実行
2. 「Kernel」→「Restart Kernel」を選択
3. 再起動後、次のセルから実行を続ける

In [ ]:
# Pillow依存関係の修正（実行後にカーネルを再起動してください）
!pip uninstall -y pillow PIL
!pip install -U "pillow==10.4.0"

# オプション: websockets警告の抑制
!pip install -U "websockets==15.0.1"

print("\n✅ 完了しました。Kernel → Restart Kernel を実行してください。")

In [ ]:
# 再起動後の確認（このセルを最初に実行）
import PIL
import PIL._util as u

print(f"Pillow version: {PIL.__version__}")  # => 10.4.0
print(f"is_directory exists: {hasattr(u, 'is_directory')}")  # => True

if PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"):
    print("✅ Pillow依存関係は正常です。次のセルに進んでください。")
else:
    print("⚠️ 問題があります。カーネルを再起動しましたか？")

In [ ]:
# 必要なパッケージのインストール
!pip install -q transformers torch opencv-python tqdm loguru iopaint

print("✅ パッケージのインストールが完了しました。")
print("\n📝 Note: ffmpegが必要です。インストールされていない場合は以下のコマンドを実行してください:")
print("   Ubuntu/Debian: sudo apt-get install -y ffmpeg")
print("   macOS: brew install ffmpeg")
print("   Windows: https://ffmpeg.org/download.html からダウンロード")

## 2. デバイス確認

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ GPUが利用できません。CPUで処理します（処理時間が長くなる可能性があります）。")

## 3. 入力動画の設定

処理する動画ファイルのパスを指定してください。

In [ ]:
import os

# 入力動画のパスを指定（絶対パスまたは相対パス）
input_video = "input.mp4"  # ここを変更してください

# ファイルの存在確認
if os.path.exists(input_video):
    abs_path = os.path.abspath(input_video)
    file_size = os.path.getsize(abs_path) / (1024 * 1024)  # MB
    print(f"✅ 入力ファイル確認: {abs_path}")
    print(f"📊 ファイルサイズ: {file_size:.2f} MB")
else:
    print(f"❌ エラー: 入力ファイルが見つかりません: {input_video}")
    print("\n💡 ヒント: 以下のいずれかの方法で指定してください:")
    print("   1. 絶対パス: input_video = '/full/path/to/video.mp4'")
    print("   2. 相対パス: input_video = './videos/input.mp4'")
    print("   3. このノートブックと同じディレクトリ: input_video = 'video.mp4'")

## 4. 動画処理

### パラメータ設定

- `max_bbox_percent`: バウンディングボックスの最大サイズ（画像に対する割合%）
- `frame_step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `target_fps`: 出力fps（0=入力と同じ）

In [ ]:
import os

# パラメータ設定（必要に応じて調整）
max_bbox_percent = 10.0  # バウンディングボックスの最大サイズ
frame_step = 1           # 1=全フレーム、2=隔フレーム
target_fps = 0.0         # 0=入力と同じ

# 出力ファイル名の設定
output_video = "output_no_watermark.mp4"

# 入力ファイルの確認
if not os.path.exists(input_video):
    print(f"❌ エラー: 入力ファイルが見つかりません: {input_video}")
    print("前のセルで入力ファイルのパスを正しく設定してください。")
else:
    input_path = os.path.abspath(input_video)
    output_path = os.path.abspath(output_video)
    
    print(f"✅ 入力ファイル: {input_path}")
    print(f"📝 出力先: {output_path}")
    print(f"⚙️ パラメータ: max_bbox_percent={max_bbox_percent}, frame_step={frame_step}, target_fps={target_fps}")
    print("\n🚀 処理を開始します...\n")
    
    # remwm.pyを実行
    !python remwm.py "{input_path}" "{output_path}" \
      --max-bbox-percent {max_bbox_percent} \
      --frame-step {frame_step} \
      --target-fps {target_fps}
    
    # 出力ファイルの確認
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"\n✅ 処理完了: {output_path}")
        print(f"📊 ファイルサイズ: {file_size:.2f} MB")
    else:
        print(f"\n❌ エラー: 出力ファイルが作成されませんでした")
        print("上記のエラーメッセージを確認してください")

## 5. 結果の確認

処理が完了した動画は、指定した出力パスに保存されています。

In [ ]:
# 出力ファイルの情報を表示
if os.path.exists(output_video):
    abs_path = os.path.abspath(output_video)
    file_size = os.path.getsize(abs_path) / (1024 * 1024)  # MB
    print(f"📁 出力ファイル: {abs_path}")
    print(f"📊 ファイルサイズ: {file_size:.2f} MB")
    
    # ビデオプレーヤーで表示（Jupyter環境で対応している場合）
    try:
        from IPython.display import Video
        print("\n📺 プレビュー:")
        display(Video(abs_path, width=640))
    except:
        print("\n💡 ビデオプレーヤーでファイルを開いて確認してください。")
else:
    print("❌ 出力ファイルが見つかりません。処理が正常に完了したか確認してください。")

## 6. 使い方のヒント

### 処理速度を上げたい場合

```python
frame_step = 2  # 隔フレーム処理（約2倍速）
```

### 出力fpsを固定したい場合

```python
target_fps = 30  # 30fpsで出力
```

### 組み合わせ例

```python
frame_step = 2
target_fps = 30
# → 隔フレーム処理して30fpsで出力
```

### バッチ処理

複数の動画を処理したい場合:

```python
import glob

# 特定のディレクトリ内の全mp4ファイルを処理
video_files = glob.glob("./videos/*.mp4")

for video in video_files:
    input_video = video
    output_video = video.replace(".mp4", "_no_watermark.mp4")
    # 処理コードを実行...
```